In [1]:
from config.settings import SRC_DIR
import os
os.chdir(SRC_DIR)

In [2]:
from config.settings import DATA_DIR
import os
from tqdm.auto import tqdm

file_dir = DATA_DIR / 'qc_data' / 'qc_templates'
files = os.listdir(file_dir)
file_names = [file[:-4] for file in files]
file_paths = [file_dir/file for file in files]   

In [ ]:
from document_processing_system.components.model_inference import ModelInference

# Initialize the model
model = ModelInference(model_name="Qwen/Qwen2.5-14B")

INFO:root:Loading model and tokenizer...
INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
import polars as pl
import os
import json
from document_processing_system.components.data_ingestion import DataIngestion
from document_processing_system.components.data_extraction import DataExtraction

for pdf, pdf_path in zip(tqdm(file_names),file_paths):

    data_ingester = DataIngestion(pdf_path)
    data_extracter = DataExtraction(data_ingester)
    pages, tables, images = data_extracter.extract_data()
    all_items = data_extracter.filter_pages(pages)

    from config.settings import DATA_DIR

    # Define the PDF folder path
    pdf_folder = os.path.join(DATA_DIR, 'qc_data', 'qc_reports_output', pdf, 'qc_reports_14b')

    # Ensure the directory exists
    os.makedirs(pdf_folder, exist_ok=True)

    response = model.process_items(all_items, output_folder=pdf_folder)

    flat_data = [item for sublist in response for item in sublist]

    data = pl.DataFrame(flat_data)

    # Save data
    output_folder = os.path.join(DATA_DIR, 'qc_data', 'qc_reports_output', pdf, 'pdf_data')

    data.write_excel(f"{output_folder}/data.xlsx")


    with open(f'{output_folder}/output.json', "w", encoding="utf-8") as json_file:
        json.dump(flat_data, json_file, indent=4, ensure_ascii=False)

  0%|          | 0/7 [00:00<?, ?it/s]

Extracting pages:   0%|          | 0/8 [00:00<?, ?pages/s]

Filtering pages:   0%|          | 0/8 [00:00<?, ?pages/s]

Extracting pages:   0%|          | 0/4 [00:00<?, ?pages/s]

Filtering pages:   0%|          | 0/4 [00:00<?, ?pages/s]

...

Filtering pages:   0%|          | 0/5 [00:00<?, ?pages/s]

Extracting pages:   0%|          | 0/15 [00:00<?, ?pages/s]

Filtering pages:   0%|          | 0/15 [00:00<?, ?pages/s]

Extracting pages:   0%|          | 0/30 [00:00<?, ?pages/s]

Filtering pages:   0%|          | 0/30 [00:00<?, ?pages/s]